In [128]:
from dotenv import load_dotenv

load_dotenv()

True

In [163]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


class Player:
    def __init__(self, model):
        self.observations = []
        self.model = model

    def ask(self, questions_left):
        template = """
        You are a player in a game where you need to ask Yes/No questions about 
        a thing and guess what it is.

        Here is what you already know about this thing:

        {observations}

        You only have {questions_left} questions left to ask. You want to guess
        the thing in as few questions as possible. If there's only 1 question left, 
        you must make a guess or you'll lose the game. Be aggresive and try to
        guess the thing as soon as possible.

        You can also ask a binary question that will help you guess the thing.
        The question must be answered with a Yes/No. No other questions are allowed.
         
        Be as concise as possible when asking a question. Do not anounce that you
        will ask the question. Do not say "Let's get started", or introduce your 
        question. Just write the question.

        If you already know what the thing is, write your guess by saying
        "My guess is <thing>." For example, if you think the thing is an apple,
        write "My guess is apple".

        Examples of good questions:

        - Is it a fruit?
        - Is it bigger than a car?
        - Is the thing alive?

        Examples of bad questions:

        - Can I ask a question?
        - Can you tell me more about the thing?
        - What is the thing?
        - How does the thing look like?
        """
        prompt = PromptTemplate.from_template(template)
        chain = prompt | self.model | StrOutputParser()
        return chain.invoke(
            {
                "observations": "\n".join(self.observations),
                "questions_left": questions_left,
            }
        )

    def add_observation(self, question, answer):
        self.observations.append(f"Question: {question}. Answer: {answer}")

In [164]:
from langchain_community.llms import Ollama

player1 = Player(model=Ollama(model="llama3"))
question = player1.ask(20)
question

'Is it a living thing?'

In [165]:
class Host:
    def __init__(self, model):
        self.model = model

    def answer(self, concept, question):
        template = """
        You are the host of a game where a player asks questions about
        a thing to guess what it is.

        The thing is: {concept}.

        The player has asked you the following question: {question}.

        If the player guessed the thing, answer with the word "GUESSED".
        Do not write anything else. Do not be pedantic and be lenient
        in your evaluation. For example, If the player guessed "PC" or 
        "laptop" and the concept is "Computer", you should answer "GUESSED".

        If the player didn't guessed the thing, answer its question with a 
        simple Yes or No. Do not say anything else. Do not use any punctuation.

        For example, if the player asks "Is the thing a red apple?", an 
        the thing is an apple, you should answer "GUESSED".

        If the player asks "Is the thing a fruit?", and the concept is 
        an apple, you should answer "Yes".
       
        """
        prompt = PromptTemplate.from_template(template)
        chain = prompt | self.model | StrOutputParser()
        return chain.invoke({"concept": concept, "question": question})

In [166]:
from langchain_openai.chat_models import ChatOpenAI

host = Host(model=Ollama(model="llama3"))
answer = host.answer("cat", question)
answer

'Yes'

In [167]:
class Game:
    def __init__(self, host_model="gpt-3.5-turbo", player_model="llama3", rounds=20):
        self.host_model = host_model
        self.player_model = player_model
        self.rounds = 20

    def start(self, concepts):
        score = 0
        for concept in concepts:
            print(f"Concept: {concept}")
            score += self._play(concept)

        print(
            f"Score: {score} out of {len(concepts)}. Accuracy: {(score / len(concepts)):.2f}"
        )

        return score

    def _play(self, concept):
        host = Host(model=self._create_model(self.host_model))
        player = Player(model=self._create_model(self.player_model))

        for r in range(self.rounds):
            question = player.ask(self.rounds - r)
            answer = host.answer(concept, question)

            print(f"Question {r + 1}: {question}. Answer: {answer}")

            player.add_observation(question, answer)

            if "guessed" in answer.lower():
                print(f"Player guessed the concept: {concept}")
                return True

        print("The player did not guess the concept.")
        return False

    def _create_model(self, model):
        if "gpt" in model:
            return ChatOpenAI(model=model)

        return Ollama(model=model)

In [168]:
concepts = [
    "cat",
    "computer",
    "house",
    "car",
    "tree",
]

In [170]:
game = Game(host_model="llama3", player_model="llama3")
game.start(concepts)

Concept: cat
Question 1: Is it man-made?. Answer: No
Question 2: Is the thing alive?. Answer: Yes
Question 3: Is it a plant?. Answer: No
Question 4: Is it an animal?. Answer: Yes
Question 5: Does it have fur?. Answer: Yes
Question 6: Does it have four legs?. Answer: Yes
Question 7: Is it a domesticated animal?. Answer: Yes
Question 8: Does it bark?. Answer: No
Question 9: Does it meow?. Answer: Yes
Question 10: My guess is cat.. Answer: GUESSED
Player guessed the concept: cat
Concept: computer
Question 1: Is it a living thing?. Answer: No
Question 2: Is it man-made?. Answer: Yes
Question 3: Is it used primarily indoors?. Answer: Yes
Question 4: Is it electronic?. Answer: Yes
Question 5: Is it used for communication?. Answer: Yes
Question 6: Is it portable?. Answer: Yes
Question 7: Does it primarily fit in the hand?. Answer: No
Question 8: Is it used primarily for accessing the internet?. Answer: Yes
Question 9: Is it a laptop?. Answer: No
Question 10: Is it a tablet?. Answer: No
Questi

1